%matplotlib inline

In [2]:

import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle
import math

import pandas as pd

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

#from scipy.misc import imread, imresize



In [3]:
train_df = pd.read_csv('../../data/Manually_Annotated_file_lists/training_face_mesh_crop.csv')
train_df.subDirectory_filePath = '../../data/Manually_Annotated_Images_FaceMesh_Cropped/' + train_df.subDirectory_filePath

#train_df_2 = pd.read_csv('../../data/Automatically_annotated_file_list/automatically_annotated_face_mesh_crop.csv')
#train_df_2.subDirectory_filePath = '../../data/Automatically_Annotated_Images_FaceMesh_Cropped/' + train_df_2.subDirectory_filePath
#train_df = train_df.append(train_df_2)
#del train_df_2

train_df = train_df[train_df['have_facemesh']]

In [4]:
class DataFrameBatchIterator:
    def __init__(self, dataframe, batch_size):
        self.df = dataframe
        self.index = 0
        self.batch_size = batch_size

    def __iter__(self):
        return self
        
    def __next__(self):
        if self.index < len(train_df):
            self.index += self.batch_size
            return self.df[self.index - self.batch_size: self.index].copy()
        else:
            raise StopIteration

In [5]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf

In [6]:
INPUT_SIZE = (224, 224, 3)

In [7]:
mobilenet = tf.keras.applications.MobileNetV2(input_shape=INPUT_SIZE,
                                           include_top=False,
                                           weights='imagenet')

In [8]:
img_inputs = tf.keras.Input(shape=(INPUT_SIZE))
mobilenet_output = mobilenet(img_inputs)

In [9]:
x = tf.keras.layers.GlobalAveragePooling2D()(mobilenet_output)
for size in [640]:
    x = tf.keras.layers.Dense(size, activation = 'relu')(x)

In [10]:
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

In [11]:
model = tf.keras.Model(inputs=img_inputs, outputs=outputs, name="mobilenet")

In [12]:
model.summary()

Model: "mobilenet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 640)               819840    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 641       
Total params: 3,078,465
Trainable params: 3,044,353
Non-trainable params: 34,112
_________________________________________________________________


In [13]:
#model.layers[1].trainable = False

In [14]:
# Instantiate an optimizer.
optimizer = tf.keras.optimizers.Adam()
# Instantiate a loss function.
loss_fn = tf.keras.losses.MeanSquaredError()

In [15]:
batch_size = 32

In [19]:
import time
start_time = time.time()

total_loss = 0
total_step = 0
for step, row in enumerate(DataFrameBatchIterator(train_df, batch_size=batch_size)):

    # Process Data
    imgs = row.subDirectory_filePath.apply(lambda x: cv2.resize(
        cv2.cvtColor(cv2.imread(x), cv2.COLOR_BGR2RGB), (224, 224)))

    img_array = np.array([])
    for img in list(imgs):
        img_array = np.append(img_array, img)

    img_array = img_array.reshape(-1, *list(imgs)[0].shape)

    y_arousal = np.array(row.arousal)

    # Train
    with tf.GradientTape() as tape:
        logits = model(img_array, training=True)
        loss_value = loss_fn(y_arousal, logits)
        
        total_loss += float(loss_value)
        total_step += 1
    break
    grads = tape.gradient(loss_value, model.trainable_weights)

    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    # Log every 200 batches.
    break
    if step % 10 == 0:
        print(
            f"Training loss at step {step}: {total_loss/total_step}"
        )
        print("Seen so far: %s samples" % ((step + 1) * batch_size))
        print(f"--- {time.time() - start_time} seconds for iter {total_step} step, each step have {batch_size}, so the model train with {total_step*batch_size} img this iter ---\n")
        total_loss = 0
        total_step = 0
        
        start_time = time.time()

In [23]:
loss_value

<tf.Tensor: shape=(), dtype=float32, numpy=1.1987042>

In [18]:
grads = tape.gradient(loss_value, model.trainable_weights)

[<tf.Tensor: shape=(3, 3, 3, 32), dtype=float32, numpy=
 array([[[[-6.90611079e-03,  2.72850879e-02, -5.49776247e-03,
            0.00000000e+00, -2.25426219e-02,  2.67511373e-03,
           -1.99420145e-03,  0.00000000e+00,  2.63181771e-03,
            1.82717331e-02,  2.83837738e-03,  0.00000000e+00,
            0.00000000e+00,  0.00000000e+00, -2.34968332e-03,
            6.32290030e-03,  0.00000000e+00, -2.97214766e-03,
           -2.04058108e-03,  0.00000000e+00,  9.24325956e-04,
           -7.11463112e-03,  2.00711563e-02, -6.82683778e-04,
            2.05215672e-03, -2.06443877e-03,  0.00000000e+00,
           -3.51995155e-02,  0.00000000e+00,  2.15792470e-03,
            2.41510081e-03, -1.85137033e-04],
          [-1.16357179e-02,  2.82840077e-02, -1.35005626e-03,
            0.00000000e+00, -1.57444496e-02,  8.28576041e-04,
            4.28195490e-04,  0.00000000e+00, -2.73122080e-03,
            1.53735010e-02, -1.07039604e-03,  0.00000000e+00,
            0.00000000e+00,  0